### 데이콘에서 제공한 변수별 제한 수치
- 시간당 내부 평균온도: 4도~40도  
- 시간당 내부 평균습도: 0% ~ 100%  
- 시간당 내부 평균 co2 농도 : 0ppm ~ 1200 ppm  
- 시간당 평균 EC : 0 ~ 8  
- 시간당 분무량 : 0 ~ 3000 / 일간 누적 분무량 0 ~ 72,000  
- 시간당 백색광량 : 0 ~ 120,000 / 일간 누적 백색광량 0 ~ 2,880,000  
- 시간당 적색광량 : 0 ~ 120,000 / 일간 누적 적색광량 0 ~ 2,880,000   
- 시간당 청색광량 : 0 ~ 120,000 / 일간 누적 청색광량 0 ~ 2,880,000  
- 시간당 총광량 : 0 ~ 120,000 / 일간 누적 총광량 0 ~ 2,880,000  

# Data Load

In [8]:
import pandas as pd

In [9]:
from modules.data_store import data_store

In [10]:
data_store = data_store()

Load the saved pickle file


In [13]:
trial = data_store.return_last_trial()
df = data_store.dict_df[trial]
df.columns

Index(['DAT', 'predicted_weight_g', 'train/test', 'file_num', 'obs_time',
       '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량',
       '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량',
       '일간누적총광량', 'sum', 'total_sum', 'weight', 'target'],
      dtype='object')

In [14]:
df[df['train/test']=='CASE'].describe()

,DAT,predicted_weight_g,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target
count,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,1.881600e+04,18816.000000,18816.000000
mean,14.500000,34.919681,26.027719,63.935090,498.860695,1.507566,523.627918,6365.916673,6315.562930,72910.240149,1571.862912,18201.358998,1241.905981,14139.184959,9129.331824,105250.784106,19372.622634,4.649429e+05,0.041667,1.454987
std,8.077962,41.580313,3.948988,11.657905,120.421248,1.042010,544.323281,6903.680044,8188.067575,86040.447105,2937.252667,34386.384982,2474.426437,27743.227265,9175.427411,97397.224913,18416.095388,1.578185e+05,0.039194,2.277581
min,1.000000,0.167719,0.000000,0.000000,60.400000,0.000000,-653.260000,0.000000,-138371.246100,0.000000,-8793.607200,0.000000,-18570.857500,0.000000,-165735.710800,0.000000,-330190.428957,3.975293e+04,-2.121948,-4.470883
25%,7.750000,4.869563,23.971250,57.494583,420.245833,0.929581,0.000000,1234.280000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7280.962850,876.121634,3.967611e+05,0.002430,0.040572
50%,14.500000,16.118144,26.291667,62.118333,473.983333,1.308517,252.000000,3360.485000,0.000000,23205.750000,0.000000,0.000000,0.000000,0.000000,9242.350000,92172.360000,19796.105613,4.422953e+05,0.047776,0.298112
75%,21.250000,51.272294,28.106667,73.157166,551.105952,2.291322,769.000000,9990.570000,18236.625400,145422.700000,1494.698100,17529.296400,1306.461000,18484.700000,18255.190000,177491.499450,37204.070917,5.356698e+05,0.075820,1.809942
max,28.000000,164.150300,42.643334,93.350002,1714.850000,5.462324,3144.130000,42855.230000,18564.600000,292014.969800,9928.800000,200561.760000,9399.000000,190016.450000,37892.400000,559488.881000,76641.128238,1.136716e+06,0.436592,12.043547


- obs_time 통일  
- 내부온도관측치 0 처리  
- 내부습도관측치 0 처리    
- 시간당백색광량, 시간당적색광량, 시간당청색광량 마이너스값 조절  
- 모든 누적광량 컬럼 cumsum 재 계산  
- 적색광,청색광 값이 0인 부분이 많은데 채워줘야하는지?   

null값 처리할건 처리하고 null값 채워주는 모듈 사용한뒤 제출해보기  
<br>

이상치 모듈 한번더 실행후 EBM해보기


In [30]:
df[df['내부온도관측치']==0]

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour
16143,1,1.656917,CASE,04,15:00,0.0,0.0,560.254237,1.436618,1550.86,...,10160.472,3565.354,21154.016,23634.2048,182458.1789,49380.960456,497502.195384,0.099258,0.164462,15


In [36]:
load_data(df,1,'CASE','08')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour
0,1,1.918119,CASE,08,00:00,27.680000,73.026667,466.750000,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,629.156667,381790.957381,0.001648,0.003161,0
1,1,1.918119,CASE,08,01:00,27.770000,73.120000,464.516667,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,627.106666,381790.957381,0.001643,0.003151,1
2,1,1.918119,CASE,08,02:00,27.766666,73.130001,466.233333,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,628.830000,381790.957381,0.001647,0.003159,2
3,1,1.918119,CASE,08,03:00,27.896666,73.488334,468.466667,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,631.551667,381790.957381,0.001654,0.003173,3
4,1,1.918119,CASE,08,04:00,27.921666,73.543333,471.550000,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,634.715000,381790.957381,0.001662,0.003189,4
5,1,1.918119,CASE,08,05:00,27.828333,73.635000,475.033333,0.0,61.7,...,0.0,0.0,0.0,0.0000,0.0000,638.196666,381790.957381,0.001672,0.003206,5
6,1,1.918119,CASE,08,06:00,28.051667,73.621668,485.683333,0.0,123.4,...,0.0,0.0,0.0,4269.8580,4269.8580,9250.472668,381790.957381,0.024229,0.046474,6
7,1,1.918119,CASE,08,07:00,29.698333,72.093334,486.900000,0.0,123.4,...,0.0,0.0,0.0,13326.2887,17596.1467,27364.669067,381790.957381,0.071674,0.137480,7
8,1,1.918119,CASE,08,08:00,31.133333,70.118334,481.516667,0.0,123.4,...,0.0,0.0,0.0,18249.0018,35845.1485,37204.171934,381790.957381,0.097446,0.186914,8
9,1,1.918119,CASE,08,09:00,30.758333,69.293332,506.300000,0.0,123.4,...,0.0,0.0,0.0,18255.1900,54100.3385,37240.131665,381790.957381,0.097541,0.187095,9


In [32]:
load_data(df,1,'CASE','04')

,DAT,predicted_weight_g,train/test,file_num,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,...,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour
16128,1,1.656917,CASE,04,00:00,16.276667,58.665000,602.633333,0.003096,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,677.578096,497502.195384,0.001362,0.002257,0
16129,1,1.656917,CASE,04,01:00,16.370000,58.231667,593.616667,0.003903,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,668.222236,497502.195384,0.001343,0.002225,1
16130,1,1.656917,CASE,04,02:00,16.286667,58.146667,584.783333,0.004276,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,659.220943,497502.195384,0.001325,0.002196,2
16131,1,1.656917,CASE,04,03:00,16.368333,57.860000,578.350000,0.004626,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,652.582959,497502.195384,0.001312,0.002173,3
16132,1,1.656917,CASE,04,04:00,16.371667,57.481667,569.966667,0.004987,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,643.824987,497502.195384,0.001294,0.002144,4
16133,1,1.656917,CASE,04,05:00,16.330000,57.521667,563.883333,0.004963,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,637.739963,497502.195384,0.001282,0.002124,5
16134,1,1.656917,CASE,04,06:00,16.686667,56.460000,560.100000,0.005228,0.00,...,188.6472,178.5810,178.5810,4569.0160,4569.0160,9771.283894,497502.195384,0.019641,0.032543,6
16135,1,1.656917,CASE,04,07:00,17.503333,53.740000,556.100000,0.005300,0.00,...,860.4960,635.9990,814.5800,14597.0073,19166.0233,29821.363233,497502.195384,0.059942,0.099319,7
16136,1,1.656917,CASE,04,08:00,19.920000,49.250526,554.583333,0.008625,0.00,...,1833.5184,1315.8600,2130.4400,20531.6960,39697.7193,41687.154485,497502.195384,0.083793,0.138838,8
16137,1,1.656917,CASE,04,09:00,24.740000,41.851069,548.750000,0.008408,0.00,...,2809.8504,2232.2625,4362.7025,21463.7845,61161.5038,43542.918476,497502.195384,0.087523,0.145019,9


- obs_time 통일성 있게 바꾸기  
- 중간중간 마이너스값 조절해주기  
- 적색광,청색광 값이 0인 부분이 많은데 채워줘야하는지?  

In [38]:
from missingpy import MissForest
imputer = MissForest()
X_imputed = imputer.fit_transform(X)

ModuleNotFoundError: No module named 'missingpy'

In [26]:
def load_data(df,DAT,train_test,file_num):
    return df[(df['DAT']==DAT)&(df['train/test']==train_test)&(df['file_num']==file_num)]

In [27]:
def get_obs_time(x):
    time = x.split(':')
    if int(time[1]) == 0:
        return int(time[0])
    else :
        return int(time[0])+1
    
df['obs_hour'] = df['obs_time'].apply(get_obs_time)
df['obs_hour'].value_counts()

15    924
14    924
16    924
1     924
17    924
2     924
18    924
3     924
19    924
4     924
20    924
5     924
21    924
6     924
22    924
7     924
23    924
8     924
9     924
10    924
11    924
12    924
13    924
0     924
Name: obs_hour, dtype: int64

In [ ]:
diff_cols  = ['diff_온도','diff_습도','diff_co2','diff_ec']
df['diff_온도']=df.groupby(['DAT','train/test','file_num']).shift(1)

In [50]:
# import modules.config as config
# input_cols = config.input_cols
input_cols = ['DAT','obs_hour','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','일간누적분무량','시간당백색광량','일간누적백색광량','시간당적색광량','일간누적적색광량','시간당청색광량','일간누적청색광량','시간당총광량','일간누적총광량']
train_df = df[df['train/test']=='CASE']
test_df = df[df['train/test']=='TEST']

from pyod.models.ecod import ECOD
clf = ECOD()
clf.fit(train_df[input_cols])

# get outlier scores
y_train_scores = clf.decision_scores_  # raw outlier scores on the train data
y_test_scores = clf.decision_function(test_df[input_cols]) 
y_train_scores

y_test_scores

array([23.19471741, 22.54412984, 22.1831165 , ..., 21.31143284,
       21.84289801, 22.67417843])

In [53]:
from pyod.models.iforest import IForest

clf = IForest()
clf.fit(train_df[input_cols])

# get outlier scores
y_train_scores = clf.decision_scores_  # raw outlier scores on the train data
y_test_scores = clf.decision_function(test_df[input_cols]) 
y_train_scores

# y_test_scores

array([-0.09776891, -0.10334189, -0.10970761, ..., -0.03308606,
       -0.0340276 , -0.02516442])

In [55]:
pd.DataFrame(y_train_scores).describe()

,0
count,18816.000000
mean,-0.061588
std,0.050535
min,-0.151399
25%,-0.098607
50%,-0.066688
75%,-0.035923
max,0.180344


In [58]:
y_test_pred = clf.predict(train_df[input_cols])
train_df['out'] = y_test_pred

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
train_df[train_df['out']==1].describe()

,DAT,predicted_weight_g,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,...,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,sum,total_sum,weight,target,obs_hour,out
count,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,...,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1.882000e+03,1882.000000,1882.000000,1882.000000,1882.0
mean,14.275239,45.367431,26.250840,61.035004,518.913964,1.847253,643.182194,12562.247221,7448.347029,94360.254757,...,4387.578387,51945.189109,16558.491608,217664.944627,34368.212471,6.235531e+05,0.054592,2.179725,17.722635,1.0
std,8.578729,54.011918,3.598700,16.135622,225.428573,1.059161,656.291764,10312.184739,8550.852593,104283.774013,...,4046.235925,49556.992854,12210.524879,124388.081121,24160.215724,3.069718e+05,0.024488,2.675606,4.477436,0.0
min,1.000000,0.258003,0.000000,0.000000,350.192983,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,501.632324,3.975293e+04,0.000522,0.000891,2.000000,1.0
25%,7.000000,4.904527,25.131101,46.917639,388.975000,1.158759,0.000000,962.990000,0.000000,0.000000,...,0.000000,5263.440000,9242.350000,147148.820000,19796.720655,4.134766e+05,0.045239,0.252368,15.000000,1.0
50%,15.000000,14.271356,26.259661,63.526667,433.316667,1.743751,407.620000,13964.715000,0.000000,53821.869500,...,3563.787500,35324.575000,9763.320000,182000.580000,21010.494247,4.509821e+05,0.050029,0.664031,19.000000,1.0
75%,22.000000,80.896230,28.174583,75.943333,550.716667,2.708414,1267.940000,20663.810000,18255.190000,197864.600900,...,9242.350000,82867.850000,23206.083125,254139.996125,48060.875548,1.060889e+06,0.066327,3.918390,21.000000,1.0
max,28.000000,164.150300,39.842373,93.000000,1714.850000,4.723398,3144.130000,42855.230000,18564.600000,292014.969800,...,9399.000000,190016.450000,37892.400000,559488.881000,76641.128238,1.136716e+06,0.142676,11.835178,23.000000,1.0


In [62]:
from interpret.glassbox import ExplainableBoostingRegressor

X_train = train_df[train_df['out']!=1][config.input_cols]
y_train = train_df[train_df['out']!=1]['target']

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['DAT', 'obs_time', '내부온도관측치',
                                            '내부습도관측치', 'co2관측치', 'ec관측치',
                                            '시간당분무량', '일간누적분무량', '시간당백색광량',
                                            '일간누적백색광량', '시간당적색광량', '일간누적적색광량',
                                            '시간당청색광량', '일간누적청색광량', '시간당총광량',
                                            '일간누적총광량', 'DAT x 시간당총광량',
                                            'DAT x 시간당백색광량', 'DAT x 일간누적총광량',
                                            'DAT x obs_time', 'DAT x 내부온도관측치',
                                            'DAT x 시간당분무량', '내부습도관측치 x 시간당총광량',
                                            'DAT x 일간누적백색광량', 'DAT x co2관측치',
                                            'DAT x 일간누적분무량'],
                             feature_...'continuous', 'categorical',
                                            'continuous', 'continuous',
                               

In [63]:
X_test = test_df[input_cols]


test_df['predicted_weight_g'] = ebm.predict(X_test)

/Users/youngyong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
base_rslt_df = test_df[['DAT','file_num','predicted_weight_g']].groupby(['DAT','file_num']).sum().reset_index()
def predict_rslt_weight(x):
    if x < 0 :
        return 0
    else : 
        return x

base_rslt_df['predicted_weight_g'] = base_rslt_df['predicted_weight_g'].apply(predict_rslt_weight)
# base_rslt_df = base_rslt_df.set_index('DAT')
base_rslt_df

,DAT,file_num,predicted_weight_g
0,1,01,8.458475
1,1,02,0.000000
2,1,03,8.883275
3,1,04,0.181287
4,1,05,3.178967
...,...,...,...
135,28,01,126.183316
136,28,02,70.447497
137,28,03,108.530320
138,28,04,131.801806


In [65]:
file_num_list = set(base_rslt_df['file_num'])
test_target_list = [] 
file_path = './open/sample_submission/'
for file_num in file_num_list:
    test_target_file = file_path+'TEST_'+file_num+'.csv'
    print(test_target_file)
    test_target_list.append(test_target_file)
    base_rslt_df[base_rslt_df['file_num']==file_num][['DAT','predicted_weight_g']].to_csv(test_target_file,index=False)
    

./open/sample_submission/TEST_02.csv
./open/sample_submission/TEST_05.csv
./open/sample_submission/TEST_04.csv
./open/sample_submission/TEST_01.csv
./open/sample_submission/TEST_03.csv
